In [1]:
import pandas as pd
import numpy as np
import os, sys, glob
from tqdm import tqdm
import datetime as dt

df_citi = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/citi_df.csv')
df_weather = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Weather.csv')
df_subway = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Subway.csv')
df_event = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Events_updated_v2.csv')
df_traffic = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Traffic.csv')
df_theater = pd.read_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/Theater.csv')


In [2]:
df_citi['DateTime']=df_citi['DateTime'].str.slice(0, 10)
df1_citi = df_citi.groupby(['DateTime', 'Zipcode', 'Month','Hour']).agg({'BikeCount': ['sum']})
df1_citi = df1_citi.reset_index()

In [20]:
# The citi data has only 361 days
len(df1_citi['DateTime'].unique())

361

In [4]:
# Some zipcodes are formatted as '12345-1234', the below script is meant to drop the suffix
df1_citi['Zipcode'] = df1_citi['Zipcode'].apply(lambda x: x.split('-')[0] if '-' in x else x)
df1_citi['Zipcode'] = df1_citi['Zipcode'].apply(lambda x: x[0] + x[2:] if len(x) ==6 else x)
df_wrongZipcode = df1_citi.loc[df1_citi['Zipcode'].str.len()>5,:]
df_wrongZipcode['Zipcode'].unique()

array([], dtype=object)

In [5]:
# Create day of week variable
df1_citi['DateTime'] = pd.to_datetime(df1_citi['DateTime'], format='%Y-%m-%d')
df1_citi['DayOfWeek'] = df1_citi['DateTime'].dt.day_name()

In [6]:
# Create a boolean variable: Holidate - 1=is public holiday, 0=not a public holiday
from datetime import date

import holidays

us_holidays = holidays.CountryHoliday('US', prov=None, state='NY',years=[2017])
df1_citi['Holiday'] = df1_citi['DateTime'].isin(us_holidays)
df1_citi.head()

,DateTime,Zipcode,Month,Hour,BikeCount,DayOfWeek,Holiday
,,,,,sum,,
0,2017-01-01,10000,January,0,3,Sunday,True
1,2017-01-01,10000,January,1,8,Sunday,True
2,2017-01-01,10000,January,2,2,Sunday,True
3,2017-01-01,10000,January,4,1,Sunday,True
4,2017-01-01,10000,January,5,1,Sunday,True


In [7]:
df1_citi.columns = df1_citi.columns.droplevel(1)

In [8]:
# Weather data: prepocessing with data type
df_weather = df_weather.loc[:,['temp','feels_like','pressure','humidity',
                               'wind_speed','weather_main','weather_description',
                               'date_ET','time_ET']]
df_weather['time_ET']=df_weather['time_ET'].str.slice(0, 2)
df_weather['time_ET']=df_weather['time_ET'].astype(int)
df_weather['date_ET']=pd.to_datetime(df_weather['date_ET'], format='%Y-%m-%d')

In [9]:
# Merge Citi and Weather data
df_citiWeather = pd.merge(df1_citi, df_weather,  how='left', left_on=['DateTime','Hour'], right_on = ['date_ET','time_ET'])
df_citiWeather=df_citiWeather.drop(['date_ET', 'time_ET'], axis=1)

In [10]:
df_citiWeather.head()

,DateTime,Zipcode,Month,Hour,BikeCount,DayOfWeek,Holiday,temp,feels_like,pressure,humidity,wind_speed,weather_main,weather_description
0,2017-01-01,10000,January,0,3,Sunday,True,44.37,35.69,1013,52,8.05,Clouds,overcast clouds
1,2017-01-01,10000,January,1,8,Sunday,True,44.64,35.49,1013,54,9.17,Clouds,overcast clouds
2,2017-01-01,10000,January,2,2,Sunday,True,44.11,34.90,1013,54,9.17,Clouds,overcast clouds
3,2017-01-01,10000,January,4,1,Sunday,True,43.47,33.42,1014,52,10.29,Clear,sky is clear
4,2017-01-01,10000,January,5,1,Sunday,True,43.11,32.18,1015,48,11.41,Clear,sky is clear


In [11]:
# Preprocess subway entrance data
df_subway['Zipcode'] = df_subway['Zipcode'].str.slice(0,5)
df_subway = df_subway.groupby(['Zipcode']).agg({'Objectid': ['count']})
df1_subway = df_subway.reset_index()
df1_subway.columns = df1_subway.columns.droplevel(1)

df1_subway.head()

,Zipcode,Objectid
0,10000,5
1,10001,43
2,10002,18
3,10003,26
4,10004,5


In [12]:
# Merge Citi + Weather + Subway entrances
df_citiWeatherSub = pd.merge(df_citiWeather,df1_subway, how = 'left',left_on = ['Zipcode'],right_on=['Zipcode'])
df_citiWeatherSub = df_citiWeatherSub.rename(columns={"Objectid":"SubwayEntranceCount"})
df_citiWeatherSub.head()

,DateTime,Zipcode,Month,Hour,BikeCount,DayOfWeek,Holiday,temp,feels_like,pressure,humidity,wind_speed,weather_main,weather_description,SubwayEntranceCount
0,2017-01-01,10000,January,0,3,Sunday,True,44.37,35.69,1013,52,8.05,Clouds,overcast clouds,5.0
1,2017-01-01,10000,January,1,8,Sunday,True,44.64,35.49,1013,54,9.17,Clouds,overcast clouds,5.0
2,2017-01-01,10000,January,2,2,Sunday,True,44.11,34.90,1013,54,9.17,Clouds,overcast clouds,5.0
3,2017-01-01,10000,January,4,1,Sunday,True,43.47,33.42,1014,52,10.29,Clear,sky is clear,5.0
4,2017-01-01,10000,January,5,1,Sunday,True,43.11,32.18,1015,48,11.41,Clear,sky is clear,5.0


In [13]:
# Preprocess event data
df_event['time'] = df_event['time'].str.slice(0,10)
df_event['time'] = pd.to_datetime(df_event['time'], format='%Y-%m-%d')
df_event = df_event.rename(columns={'time':'date'})
df_event = df_event.drop(['Unnamed: 0'],axis=1)
df_event = df_event.astype({'Zipcode':str})

df_event.head()

,Zipcode,date,Special Event,Construction,Farmers Market,Sidewalk Sale,Parade,Production Event,Street Event,Plaza Event,...,Theater Load in and Load Outs_01,Miscellaneous_01,Embargo_01,Street Festival_01,Athletic Race / Tour_01,Single Block Festival_01,Stickball_01,Weekend Walk_01,Shooting Permit_01,hour
0,10000,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,10000,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,10000,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,10000,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [14]:
# Merge citi + weather + subway + event
df_citiWeatherSubEvent = pd.merge(df_citiWeatherSub,df_event, how='left',left_on=['DateTime','Zipcode','Hour'],right_on =['date','Zipcode','hour'])
df_citiWeatherSubEvent = df_citiWeatherSubEvent.drop(['date','hour'],axis=1)

df_citiWeatherSubEvent.head()

,DateTime,Zipcode,Month,Hour,BikeCount,DayOfWeek,Holiday,temp,feels_like,pressure,...,Religious Event_01,Theater Load in and Load Outs_01,Miscellaneous_01,Embargo_01,Street Festival_01,Athletic Race / Tour_01,Single Block Festival_01,Stickball_01,Weekend Walk_01,Shooting Permit_01
0,2017-01-01,10000,January,0,3,Sunday,True,44.37,35.69,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01,10000,January,1,8,Sunday,True,44.64,35.49,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01,10000,January,2,2,Sunday,True,44.11,34.90,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01,10000,January,4,1,Sunday,True,43.47,33.42,1014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01,10000,January,5,1,Sunday,True,43.11,32.18,1015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Preprocess theater data, get the no. of theater per Zip Code
df_theater = df_theater.groupby(['ZIP']).agg({'the_geom':['count']}).reset_index()
df_theater.columns = df_theater.columns.droplevel(1)
df_theater.head()

,ZIP,the_geom
0,0,1
1,10001,4
2,10002,2
3,10003,13
4,10009,3


In [16]:
# The missing zipcode is 10024, I googled it.
df_theater = df_theater.rename(columns={'the_geom':'no_theater'})
df_theater.iloc[0,0] = '10024'

In [17]:
# Merge citi + weather + subway + event + theater
df_citiWeatherSubEventTheater = pd.merge(df_citiWeatherSubEvent,df_theater, how='left',left_on=['Zipcode'],right_on =['ZIP'])
df_citiWeatherSubEventTheater['Zipcode'] = df_citiWeatherSubEventTheater['Zipcode'].astype(int)
df_citiWeatherSubEventTheater.drop('ZIP', axis=1, inplace=True)
df_citiWeatherSubEventTheater['no_theater'] = df_citiWeatherSubEventTheater['no_theater'].fillna(0) 
df_citiWeatherSubEventTheater.head()

,DateTime,Zipcode,Month,Hour,BikeCount,DayOfWeek,Holiday,temp,feels_like,pressure,...,Theater Load in and Load Outs_01,Miscellaneous_01,Embargo_01,Street Festival_01,Athletic Race / Tour_01,Single Block Festival_01,Stickball_01,Weekend Walk_01,Shooting Permit_01,no_theater
0,2017-01-01,10000,January,0,3,Sunday,True,44.37,35.69,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01,10000,January,1,8,Sunday,True,44.64,35.49,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01,10000,January,2,2,Sunday,True,44.11,34.90,1013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01,10000,January,4,1,Sunday,True,43.47,33.42,1014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01,10000,January,5,1,Sunday,True,43.11,32.18,1015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_traffic = df_traffic.loc[:,['Date','Time','Zipcode','Count']]


In [19]:
# Preprocess the traffic data
hourMapping = {'11:00-12:00AM' : 23,
               '4:00-5:00AM' : 4,
                '7:00-8:00AM':7,
                '9:00-10:00PM':21,
                '10:00-11:00AM':10,
                '5:00-6:00AM':5,
                '12:00-1:00PM':12,
                '4:00-5:00PM':16,
                '12:00-1:00 AM':0,
                '7:00-8:00PM':19,
                '2:00-3:00PM':14,
                '3:00-4:00AM':3,
                '11:00-12:00PM':11,
                '6:00-7:00PM':18,
                '1:00-2:00PM':13,
                '9:00-10:00AM':9,
                '6:00-7:00AM':6,
                '3:00-4:00PM':15,
                '8:00-9:00PM':20,
                 '8:00-9:00AM':8,
                '2:00-3:00AM':2,
                '1:00-2:00AM':1,
                '10:00-11:00PM':22,
                '5:00-6:00PM':17}
    
df_traffic['Hour'] = df_traffic['Time'].map(hourMapping)
df_traffic['DateTime'] = pd.to_datetime(df_traffic['Date'])
df_traffic.head()

,Date,Time,Zipcode,Count,Hour,DateTime
0,1/14/17,12:00-1:00 AM,10036,1286,0,2017-01-14
1,1/14/17,1:00-2:00AM,10036,1061,1,2017-01-14
2,1/14/17,2:00-3:00AM,10036,714,2,2017-01-14
3,1/14/17,3:00-4:00AM,10036,531,3,2017-01-14
4,1/14/17,4:00-5:00AM,10036,422,4,2017-01-14


In [21]:
# !!!The traffic data is not for full year, large volume of missing values!!!!
# INNER JOIN all table
df_citiWeatherSubEventTheaterTraffic = pd.merge(df_citiWeatherSubEventTheater,
                                                df_traffic[['DateTime','Hour','Zipcode', 'Count']].rename({'Count':'TrafficCount'}, axis=1),
                                                on = ['DateTime','Hour','Zipcode'])

In [22]:
print(df_citiWeatherSubEventTheater.shape,
      df_citiWeatherSubEventTheaterTraffic.shape)

print( len(df_citiWeatherSubEventTheater['DateTime'].value_counts()) )
print( len(df_citiWeatherSubEventTheaterTraffic['DateTime'].value_counts()) )

(582410, 54) (43234, 55)
361
89


In [23]:
df_citiWeatherSubEventTheater.to_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/citi_joined_Full_wo_Traffic.csv')
df_citiWeatherSubEventTheater.to_csv('/Users/graceli/Documents/Baruch MSBA/S3/BUS9430/Data/FinalData/citi_INNER_joined_Traffic.csv')